Wojciech Rabiza 157962; Wojciech Pogorzelski 158111

<font size ="5">Temat projektu: regulacja temperatury powietrza w pomieszczeniu przy pomocy regulatora PID</font>

Opis projektu: celem projektu jest zaprojektowanie układu automatycznej regulacji grzejnika w pomiesczeniu w formie aplikacji internetowej. Pod uwagę brane są siły grzewcze grzejnika, temperatura otoczenia, izolacja pomieszczenia

<font size ="5">wzory: </font>

<font size ="4">


- wzór na ciśnienie:</font> $$\Large P = P_0 * e^{\frac{-g*M*h}{R*T_{\text{room}}}}$$

P - ciśnienie

P_0 - ciśnienie na poziomie morza

g - przyspieszenie ziemskie

M - masa molowa powietrza

h - wysokość nad poziomem morza

R - stała gazowa #J/molK

T_room - temperatura pokoju

<font size ="4">

- wzór na gęstość powietrza:</font>
$$\Large  p = \frac{P}{R*T_{\text{room}}}$$

p - gęstość powietrza

R - stała gazowa #J/kgK


<font size ="4">

- wzór na masę powietrza w pomieszczeniu:</font>
$$\Large m = p*V$$ 

m - masa powietrza w pomieszczeniu

V - objętość pomieszczenia

<font size ="4">

- równanie róniczkowe temperatury pomieszczenia:</font> $$\Large \frac{dT_{\text{room}}(t)}{dt} = \frac{1}{m*Hc} \left( P_{\text{heater}}*u(t) - Ht*A(T_{\text{room}}(t) - T_{\text{outside}}(t)) \right)$$


<font size ="4">

- równanie różnicowe temperatury pomieszczenia:</font> $$\Large \frac{T_{\text{room}}(n)}{T{p}} = \frac{1}{m*Hc} \left( P_{\text{heater}}*u(n) - H*tA(T_{\text{room}}(n) - T_{\text{outside}}(n)) \right)$$

<font size ="4">

- rozwiązanie równania różnicowego rekurencja:</font>

$$\Large 
\left\{ 
 \begin{array}{ll} 
  T(0) = T0\\
  T_{\text{room}}(n+1) = \frac{Tp}{m*Hc} \left( P_{\text{heater}}*u(n) - Ht*A(T_{\text{room}}(n) - T_{\text{outside}}(n)) \right) + T_{\text{room}}(n)
 \end{array}
\right.
$$
T_outside - temperatura otoczenia

Hc - specyficzna pojemność cieplna powietrza

Ht - współczynnik przenikania ciepła

A - powierzchnia pomieszczenia

P_heater - moc grzejnika

Tp - czas próbkowania

u - wartość sterowania








In [85]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

krawedz_pokoju = 5
temperatura_docelowa = 20
czas_trwania_symulacji = 604800
wysokosc = 50
moc_grzejnika = 1200
automatyka(krawedz_pokoju, temperatura_docelowa, czas_trwania_symulacji, wysokosc, moc_grzejnika)
#interact(automatyka, krawedz_pokoju=widgets.IntSlider(min=1, max=10, step=1, value=5), temperatura_docelowa=widgets.IntSlider(min=18, max=30, step=1, value=20), czas_trwania_symulacji=widgets.IntSlider(min=1440, max=604800, step=1, value=604800), wysokosc=widgets.IntSlider(min=1, max=1000, step=1, value=50), moc_grzejnika=widgets.IntSlider(min=1000, max=2000, step=1, value=1200))


In [86]:
import numpy as np
import plotly.graph_objects as go

def automatyka(krawedz_pokoju,temperatura_docelowa,czas_trwania_symulacji,wysokosc,moc_grzejnika):
    #user inputs
    room_edge = krawedz_pokoju;
    desired_temp = temperatura_docelowa;
    simulation_duration = czas_trwania_symulacji; #minutes
    alitiude = wysokosc; #meters over sea level
    P_heater = moc_grzejnika#Watts

    #Constants
    outside_temp_amplitude = np.random.uniform(-5, 15)  # Amplitude of outside temperature change
    heat_transfer_coefficient = 0.4 #W/m^2K
    s_heat_capacity = 1012 #J/kgK
    measurement_interval = 60 #seconds


    R_d = 287.058 #J/kgK gas constant for dry air
    R_ideal = 8.314 #J/molK ideal gas constant
    air_pressure_at_sea_level = 1013 #hPa
    air_molar_mass = 0.0289647 #kg/mol
    g_acceleration = 9.80665 #m/s^2

    #Calculated constants
    room_volume = room_edge**3
    room_surface_area = 5*(room_edge**2)

    # Calculate number of measurements
    num_measurements = int(simulation_duration / measurement_interval)

    # Initialize arrays to store data
    time = np.arange(0, num_measurements)
    room_temperature = np.zeros(num_measurements)
    heat_given = np.zeros(num_measurements)
    outside_temperature = np.zeros(num_measurements)


    # Initialize room temperature to be the same as outside temperature
    outside_temperature[0] = outside_temp_amplitude * np.sin(2 * np.pi * time[0] / (24 * 60 * 60))
    room_temperature[0] = outside_temperature[0]

    #PID gains Ziegler-Nichols method
    Kp = 0.6
    Ki = 0.0008
    Kd = 108
    integral = 0
    previous_error = 0
    umax = 1
    umin = 0

    #simulate temperature control
    for i in range(1, num_measurements):
        # Calculate outside temperature
        outside_temperature[i] = outside_temp_amplitude * np.sin(2 * np.pi * time[i] / (24 * 60 ))

        #Calculate error
        error = desired_temp - room_temperature[i-1]

        #Proportional term
        P = Kp * error

        #Integral term
        integral += Ki * error * measurement_interval

        #Derivative term
        derivative = (error - previous_error) / measurement_interval

        #PID output
        u = P + integral + (Kd * derivative)

        # Calculate heat given
        heat_given[i] = P_heater * min(max(u, umin), umax)

        # Calculate air pressure
        air_pressure = air_pressure_at_sea_level * np.exp(-g_acceleration * air_molar_mass * alitiude / (R_ideal * (room_temperature[i-1] + 273.15)))

        # Calculate air density
        air_denstiy = (air_pressure / (R_d * (room_temperature[i-1] + 273.15)))*100

        # Calculate air mass
        air_mass = air_denstiy * room_volume

        # Calculate room tempreture
        room_temperature[i] = (measurement_interval / (air_mass * s_heat_capacity)) * (heat_given[i] - room_surface_area * heat_transfer_coefficient * (room_temperature[i-1] - outside_temperature[i-1] )) + room_temperature[i-1]

        # Update previous error
        previous_error = error

    fig = go.Figure(data=go.Scatter(x=time, y=room_temperature))
    fig.update_layout(title='Room temperature over time', xaxis_title='Time (minutes)', yaxis_title='Temperature (C)')
    fig.show()

    fig2 = go.Figure(data=go.Scatter(x=time, y=heat_given))
    fig2.update_layout(title='Heat given over time', xaxis_title='Time (minutes)', yaxis_title='Heat (W)')
    fig2.show()

    fig3 = go.Figure(data=go.Scatter(x=time, y=outside_temperature))
    fig3.update_layout(title='Outside temperature over time', xaxis_title='Time (minutes)', yaxis_title='Temperature (C)')
    fig3.show()


